In [34]:
import pandas as pd
import re
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [35]:
#load, clean, and merge data
demo_df = pd.read_csv('Demographic_New_York_withborough_census_trackt.csv')
demo_df['census tract'] = demo_df['Geographic Area Name'].apply(lambda x: re.findall(r'\d+', x)[0])
demo_df['census tract'] = demo_df['census tract'].astype(int)

demo_df.columns = demo_df.columns.str.replace('!!Estimate!!', ' ').str.strip()
demo_df['poverty_rate'] = demo_df['Percent below poverty level Population for whom poverty status is determined']

tree_df = pd.read_csv('DecisionTree_RandomForest_Health_Predictions.csv')
merged_df = pd.merge(tree_df, demo_df, on='census tract')

label_encoder = LabelEncoder()
merged_df['poverty_rate'] = pd.to_numeric(merged_df['poverty_rate'], errors='coerce')
merged_df['health_encoded_dt'] = label_encoder.fit_transform(merged_df['dt_predicted_health'])
merged_df['health_encoded_rf'] = label_encoder.fit_transform(merged_df['rf_predicted_health'])

In [36]:
X = merged_df[['health_encoded_dt','health_encoded_rf']]
y = merged_df['poverty_rate']

bins = [0,6,13,100]
labels = ['Low', 'Medium', 'High']
merged_df['poverty_category'] = pd.cut(merged_df['poverty_rate'], bins=bins, labels=labels)

valid_indices = merged_df[['health_encoded_dt', 'health_encoded_rf', 'poverty_category']].dropna().index
X = merged_df.loc[valid_indices, ['health_encoded_dt', 'health_encoded_rf']]
y = merged_df.loc[valid_indices, 'poverty_category']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

importances = clf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

y_pred_all = clf.predict(X)
merged_df.loc[valid_indices, 'dtfr_predicted_poverty'] = y_pred_all

merged_df.to_csv('DecisionTree_Socioeconomic_Predictions.csv', index=False)


Accuracy: 0.6011732605729877


c:\Users\archi_h340w2d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\archi_h340w2d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\archi_h340w2d\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

Classification Report:
               precision    recall  f1-score   support

        High       0.60      1.00      0.75     22033
         Low       0.00      0.00      0.00      4857
      Medium       0.00      0.00      0.00      9760

    accuracy                           0.60     36650
   macro avg       0.20      0.33      0.25     36650
weighted avg       0.36      0.60      0.45     36650

             Feature  Importance
1  health_encoded_rf    0.616699
0  health_encoded_dt    0.383301
